In [86]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.impute import KNNImputer
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
conns = f'postgres://localhost:5432/movie_recommender'

In [4]:
from sqlalchemy import create_engine

db = create_engine(conns, encoding='latin1', echo=False)

In [55]:
query = """SELECT m.title, r.rating, r.user_id
FROM movies as m
join ratings as r ON r.movie_id = m.movie_id
-- WHERE r.user_id < 50;"""

df = pd.read_sql(query, db)

In [68]:
R = df.pivot_table(values="rating", index="user_id", columns="title")

In [74]:
R

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [97]:
# Filling NaNs
imputer = KNNImputer(n_neighbors=3)

In [135]:
R_filled = pd.DataFrame(imputer.fit_transform(R).round(1), columns = R.columns, index = R.index)

In [136]:
R_filled

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.3,...,1.5,4.0,3.7,3.0,3.0,3.8,2.7,1.8,4.0,1.0
2,4.0,4.0,3.5,5.0,4.0,1.5,3.5,3.0,3.8,3.0,...,1.5,4.0,3.7,3.0,3.0,4.2,2.7,2.0,2.7,1.0
3,4.0,4.0,3.5,5.0,4.0,1.5,2.8,3.0,2.3,2.8,...,1.5,4.7,3.7,3.0,3.0,3.3,2.8,2.0,2.3,1.0
4,4.0,4.0,3.5,5.0,4.0,1.5,3.7,3.0,4.2,3.0,...,1.5,3.5,3.7,3.0,3.0,3.5,2.7,2.0,3.0,1.0
5,4.0,4.0,3.5,5.0,4.0,1.5,2.3,3.0,3.3,3.0,...,1.5,4.0,3.7,3.0,3.0,4.5,2.5,1.7,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,4.0,4.0,3.5,5.0,4.0,1.5,2.8,3.0,4.2,3.3,...,1.5,3.3,3.7,3.0,3.0,4.0,3.8,2.3,3.3,1.0
607,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.5,...,1.5,3.8,3.7,3.0,3.0,4.2,3.7,2.0,3.5,1.0
608,4.0,4.0,3.5,5.0,4.0,1.5,3.2,3.0,4.5,3.2,...,1.5,4.0,3.7,3.0,3.0,4.5,3.5,1.8,3.3,1.0


In [115]:
#cosine similarities

similarities = pd.DataFrame(cosine_similarity(R_filled), columns=R.index, index=R.index)
similarities

user_id,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.993175,0.988217,0.989478,0.991932,0.993565,0.992201,0.994081,0.994730,0.990477,...,0.995458,0.993793,0.988116,0.993186,0.992235,0.992305,0.993724,0.990002,0.991244,0.994244
2,0.993175,1.000000,0.990868,0.991287,0.993202,0.992711,0.992577,0.993252,0.993376,0.991779,...,0.993570,0.992890,0.990546,0.993567,0.993317,0.993938,0.992869,0.991720,0.993392,0.993266
3,0.988217,0.990868,1.000000,0.989248,0.991123,0.989290,0.990145,0.989801,0.989531,0.989581,...,0.989031,0.990084,0.988468,0.989995,0.991944,0.989670,0.988947,0.989365,0.992006,0.989928
4,0.989478,0.991287,0.989248,1.000000,0.992069,0.990485,0.992803,0.992256,0.990966,0.991661,...,0.990122,0.991586,0.989656,0.992089,0.992137,0.991838,0.991141,0.988109,0.992004,0.990278
5,0.991932,0.993202,0.991123,0.992069,1.000000,0.991726,0.993145,0.993844,0.992868,0.991155,...,0.992696,0.993538,0.990990,0.993467,0.992670,0.993319,0.992453,0.990811,0.994368,0.992328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.992305,0.993938,0.989670,0.991838,0.993319,0.991902,0.992936,0.994245,0.993219,0.991636,...,0.993508,0.993465,0.991512,0.994650,0.993734,1.000000,0.992787,0.992066,0.993800,0.993453
607,0.993724,0.992869,0.988947,0.991141,0.992453,0.994226,0.993373,0.993687,0.993120,0.991879,...,0.994346,0.993496,0.989207,0.994085,0.993255,0.992787,1.000000,0.990459,0.992454,0.993208
608,0.990002,0.991720,0.989365,0.988109,0.990811,0.989322,0.990197,0.991469,0.990517,0.989032,...,0.991564,0.991849,0.988535,0.991039,0.992023,0.992066,0.990459,1.000000,0.991798,0.990649


In [138]:
movies = R_filled.columns

In [140]:
user_response = {'genre': 'comedy', 'film_name1': 'Interstellar (2014)', 'rating1': '5', 'film_name2': 'Blade Runner (1982)', 'rating2': '4.8', 'film_name3': "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", 'rating3': '0.5'}

In [141]:
user_input = {user_response["film_name1"]: float(user_response["rating1"]), user_response["film_name2"]: float(user_response["rating2"]), user_response["film_name3"]: float(user_response["rating3"])}

In [142]:
user_input

{'Interstellar (2014)': 5.0,
 'Blade Runner (1982)': 4.8,
 "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)": 0.5}

In [143]:
pd.DataFrame(user_input, index=["user_input"])

,Interstellar (2014),Blade Runner (1982),Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
user_input,5.0,4.8,0.5


In [144]:
# Append new user to the user-item matrix
R_new_user = R_filled.append(pd.DataFrame(user_input, index=["user_input"]))
R_new_user

,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
1,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.3,...,1.5,4.0,3.7,3.0,3.0,3.8,2.7,1.8,4.0,1.0
2,4.0,4.0,3.5,5.0,4.0,1.5,3.5,3.0,3.8,3.0,...,1.5,4.0,3.7,3.0,3.0,4.2,2.7,2.0,2.7,1.0
3,4.0,4.0,3.5,5.0,4.0,1.5,2.8,3.0,2.3,2.8,...,1.5,4.7,3.7,3.0,3.0,3.3,2.8,2.0,2.3,1.0
4,4.0,4.0,3.5,5.0,4.0,1.5,3.7,3.0,4.2,3.0,...,1.5,3.5,3.7,3.0,3.0,3.5,2.7,2.0,3.0,1.0
5,4.0,4.0,3.5,5.0,4.0,1.5,2.3,3.0,3.3,3.0,...,1.5,4.0,3.7,3.0,3.0,4.5,2.5,1.7,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.5,...,1.5,3.8,3.7,3.0,3.0,4.2,3.7,2.0,3.5,1.0
608,4.0,4.0,3.5,5.0,4.0,1.5,3.2,3.0,4.5,3.2,...,1.5,4.0,3.7,3.0,3.0,4.5,3.5,1.8,3.3,1.0
609,4.0,4.0,3.5,5.0,4.0,1.5,3.7,3.0,3.5,3.0,...,1.5,4.0,3.7,3.0,3.0,4.2,2.3,2.3,3.5,1.0
610,4.0,4.0,3.5,5.0,4.0,1.5,3.2,3.0,3.5,3.3,...,1.5,4.0,3.5,3.0,3.0,3.5,2.0,1.5,3.5,1.0


In [145]:
# Fill the NaNs for new users
R_new_user_filled = R_new_user.fillna(2.5)
R_new_user_filled

,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
1,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.3,...,1.5,4.0,3.7,3.0,3.0,3.8,2.7,1.8,4.0,1.0
2,4.0,4.0,3.5,5.0,4.0,1.5,3.5,3.0,3.8,3.0,...,1.5,4.0,3.7,3.0,3.0,4.2,2.7,2.0,2.7,1.0
3,4.0,4.0,3.5,5.0,4.0,1.5,2.8,3.0,2.3,2.8,...,1.5,4.7,3.7,3.0,3.0,3.3,2.8,2.0,2.3,1.0
4,4.0,4.0,3.5,5.0,4.0,1.5,3.7,3.0,4.2,3.0,...,1.5,3.5,3.7,3.0,3.0,3.5,2.7,2.0,3.0,1.0
5,4.0,4.0,3.5,5.0,4.0,1.5,2.3,3.0,3.3,3.0,...,1.5,4.0,3.7,3.0,3.0,4.5,2.5,1.7,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,4.0,4.0,3.5,5.0,4.0,1.5,4.0,3.0,4.3,3.5,...,1.5,3.8,3.7,3.0,3.0,4.2,3.7,2.0,3.5,1.0
608,4.0,4.0,3.5,5.0,4.0,1.5,3.2,3.0,4.5,3.2,...,1.5,4.0,3.7,3.0,3.0,4.5,3.5,1.8,3.3,1.0
609,4.0,4.0,3.5,5.0,4.0,1.5,3.7,3.0,3.5,3.0,...,1.5,4.0,3.7,3.0,3.0,4.2,2.3,2.3,3.5,1.0
610,4.0,4.0,3.5,5.0,4.0,1.5,3.2,3.0,3.5,3.3,...,1.5,4.0,3.5,3.0,3.0,3.5,2.0,1.5,3.5,1.0


In [146]:
### Create a filter for the missing movies
# ~is basically saying "not": turns the boolean values around
movie_filter = ~R_new_user.isna().any().values
movie_filter

array([False, False, False, ..., False, False, False])

In [147]:
# Create an updated user list
updated_users = R_new_user.index
updated_users

Index([           1,            2,            3,            4,            5,
                  6,            7,            8,            9,           10,
       ...
                602,          603,          604,          605,          606,
                607,          608,          609,          610, 'user_input'],
      dtype='object', length=611)

In [148]:
R_new_user.transpose()[movie_filter].transpose()

,Blade Runner (1982),Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),Interstellar (2014)
1,4.3,4.8,4.5
2,2.8,3.5,3.0
3,4.7,4.0,4.2
4,4.0,4.0,3.8
5,3.7,4.5,3.7
...,...,...,...
607,3.7,5.0,5.0
608,3.5,3.0,4.7
609,3.8,4.5,4.7
610,5.0,4.0,3.5


In [149]:
# Based on new user's ratings, we want to calculate a similarity to the other users
similarities_new_user = pd.DataFrame(cosine_similarity(R_new_user.transpose()[movie_filter].transpose()), \
                                    index=updated_users, columns=updated_users)

In [150]:
similarities_new_user

,1,2,3,4,5,6,7,8,9,10,...,602,603,604,605,606,607,608,609,610,user_input
1,1.000000,0.998746,0.993744,0.998812,0.998423,0.999672,0.994637,0.997385,0.996686,0.858948,...,0.998959,0.984580,0.975941,0.984157,0.989183,0.994935,0.977872,0.997682,0.984193,0.833759
2,0.998746,1.000000,0.987484,0.995779,0.999649,0.998802,0.989656,0.992958,0.991538,0.863954,...,0.996145,0.985508,0.965725,0.977017,0.982315,0.995737,0.968818,0.996273,0.978203,0.805495
3,0.993744,0.987484,1.000000,0.997771,0.988528,0.991443,0.992346,0.999210,0.999413,0.865987,...,0.994189,0.965718,0.993933,0.983274,0.998910,0.980073,0.980546,0.988400,0.994936,0.878235
4,0.998812,0.995779,0.997771,1.000000,0.996359,0.997398,0.993721,0.999609,0.999055,0.870184,...,0.997772,0.976225,0.985287,0.982771,0.995145,0.989237,0.977638,0.994345,0.991382,0.848890
5,0.998423,0.999649,0.988528,0.996359,1.000000,0.997827,0.987478,0.993587,0.991884,0.876785,...,0.994958,0.980675,0.968784,0.973259,0.984554,0.992961,0.964908,0.994151,0.981898,0.802980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,0.994935,0.995737,0.980073,0.989237,0.992961,0.997165,0.993122,0.986560,0.986287,0.813905,...,0.995677,0.996924,0.952296,0.986493,0.970919,1.000000,0.979276,0.998739,0.961596,0.816113
608,0.977872,0.968818,0.980546,0.977638,0.964908,0.979750,0.994249,0.980432,0.983531,0.751803,...,0.986357,0.979601,0.965252,0.999221,0.971774,0.979276,1.000000,0.985548,0.956492,0.913699
609,0.997682,0.996273,0.988400,0.994345,0.994151,0.998992,0.997438,0.992925,0.993022,0.822762,...,0.999005,0.993683,0.965848,0.991246,0.980842,0.998739,0.985548,1.000000,0.972054,0.840094
610,0.984193,0.978203,0.994936,0.991382,0.981898,0.979490,0.975293,0.992816,0.991848,0.908181,...,0.981157,0.939576,0.996050,0.960021,0.998310,0.961596,0.956492,0.972054,1.000000,0.857829


In [151]:
# Predict ratings for new_user
similarities_user_input = similarities_new_user['user_input'][~(similarities_new_user.index=='user_input')]
similarities_user_input

1      0.833759
2      0.805495
3      0.878235
4      0.848890
5      0.802980
         ...   
606    0.876782
607    0.816113
608    0.913699
609    0.840094
610    0.857829
Name: user_input, Length: 610, dtype: float64

In [152]:
# Calculate rating predictions
rating_predictions = pd.DataFrame(\
                        np.dot(similarities_user_input, R_filled) \
                        /similarities_user_input.sum(), \
                        index=R.columns)
rating_predictions

,0
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),3.999943
...,...
eXistenZ (1999),3.811635
xXx (2002),2.903241
xXx: State of the Union (2005),2.093085


In [195]:
# Calculate recommendations by filtering for movies the user has already seen
movie_filter # we have our filter already
rating_predictions[~movie_filter].sort_values(by=0, ascending=False).head(5).index.tolist()

['Scooby-Doo! and the Loch Ness Monster (2004)',
 'The Big Bus (1976)',
 'The Adventures of Sherlock Holmes and Doctor Watson: The Treasures of Agra (1983)',
 'What Happened Was... (1994)',
 'The Adventures of Sherlock Holmes and Doctor Watson: King of Blackmailers (1980)']

In [194]:
recommended_movies.index.tolist()

['Scooby-Doo! and the Loch Ness Monster (2004)',
 'The Big Bus (1976)',
 'The Adventures of Sherlock Holmes and Doctor Watson: The Treasures of Agra (1983)',
 'What Happened Was... (1994)',
 'The Adventures of Sherlock Holmes and Doctor Watson: King of Blackmailers (1980)']

In [ ]:
new_query = """SELECT m.title, r.rating, r.user_id
FROM movies as m
join ratings as r ON r.movie_id = m.movie_id
-- WHERE r.user_id < 50;"""

df = pd.read_sql(query, db)